In [1]:
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
assert float(tf.__version__[:3]) >= 2.3

from utils.helpers import get_dataset_info
from utils.data_generator import ImageDataGenerator

from builders import builder

## Quantization Model
#### 참조사이트
- [훈련 후 정수 양자화](https://www.tensorflow.org/lite/performance/post_training_integer_quant?hl=ko)
- [TensorFlow Lite 8bits 양자화 사양](https://www.tensorflow.org/lite/performance/quantization_spec)
- [딥러닝의 Quantization (양자화)와 Quantization Aware Training](https://gaussian37.github.io/dl-concept-quantization/)

In [2]:
weights_list = os.listdir('./weights/')
weights_list

['UNet_based_on_MobileNetV2_gamma_2.h5',
 'UNet_based_on_MobileNetV2_gamma_5.h5',
 'UNet_based_on_MobileNetV2_CE.h5',
 'UNet_based_on_VGG16_29.h5',
 'UNet_based_on_MobileNetV2_QAT.h5',
 'PAN_based_on_MobileNetV2_29.h5',
 'DeepLabV3Plus_based_on_MobileNetV2_29.h5',
 'UNet_based_on_MobileNetV2_CE_QAT_288384.h5',
 'UNet_based_on_MobileNetV2_CE_QAT_288384_aug.h5',
 'UNet_based_on_MobileNetV2_CE_QAT_288384_50000.h5']

In [5]:
num_classes = 2
size = (288,384)
batch_size = 32
model_name = 'UNet'
base_model = 'MobileNetV2'
layers = tf.keras.layers
inputs = layers.Input(shape=size+(3,),dtype=tf.float32)

working_dir = os.getcwd()

quantize_model = tfmot.quantization.keras.quantize_model

model, base_model = builder(num_classes, size, model_name, base_model)
model = quantize_model(model)

# 가중치 로드
model.load_weights(os.path.join(working_dir,'weights',weights_list[-1]))

x = model(inputs)
outputs = layers.Softmax(axis=-1)(x) 

model = tf.keras.Model(inputs=inputs,outputs=outputs)

c:\Users\hci\anaconda3\envs\tensorflow\lib\site-packages\keras\layers\wrappers.py:64: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config = {'layer': generic_utils.serialize_keras_object(self.layer)}
c:\Users\hci\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [6]:
# 동적 양자화
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\hci\AppData\Local\Temp\tmp6ptw2t8p\assets


INFO:tensorflow:Assets written to: C:\Users\hci\AppData\Local\Temp\tmp6ptw2t8p\assets


In [14]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.float32'>
output:  <class 'numpy.float32'>


In [7]:
# 모델 저장
import pathlib

tflite_models_dir = pathlib.Path("./tflite_models/unet_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the quantized model:
tflite_model_quant_file = tflite_models_dir/"unet_tflite_quant_288384_50000_models.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

12931768

## Tensorflow lite Metadata Writer

In [8]:
from tflite_support.metadata_writers import image_segmenter
from tflite_support.metadata_writers import writer_utils

In [12]:
tflite_models_dir = './tflite_models/unet_tflite_models'
tflite_model_list = os.listdir(tflite_models_dir)
tflite_model_list

['unet_tflite_models.tflite',
 'unet_tflite_quant_models.tflite',
 'metadata',
 'unet_tflite_quant_qat_models.tflite',
 'unet_tflite_quant_320480_models.tflite',
 'unet_tflite_quant_288384_models.tflite',
 'unet_tflite_quant_288384_aug_models.tflite',
 'unet_tflite_quant_288384_50000_models.tflite',
 'desktop.ini']

In [13]:
ImageSegmenterWriter = image_segmenter.MetadataWriter
tflite_model_name = tflite_model_list[-2]
_MODEL_PATH = os.path.join(tflite_models_dir,tflite_model_name)

# Task Library expects label files that are in the same format as the one below.
_LABEL_FILE = "./label.txt"

model_file_name = "unet_tflite_quant_qat_288384_models_50000_metadata.tflite"
_SAVE_TO_PATH = os.path.join(tflite_models_dir,'metadata',model_file_name)

# Normalization parameters is required when reprocessing the image. It is
# optional if the image pixel values are in range of [0, 255] and the input
# tensor is quantized to uint8. See the introduction for normalization and
# quantization parameters below for more details.
# https://www.tensorflow.org/lite/models/convert/metadata#normalization_and_quantization_parameters)
_INPUT_NORM_MEAN = 127.5
_INPUT_NORM_STD = 127.5

# Create the metadata writer.
writer = ImageSegmenterWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), [_INPUT_NORM_MEAN], [_INPUT_NORM_STD],[_LABEL_FILE])

# Verify the metadata generated by metadata writer.
print(writer.get_metadata_json())

# Populate the metadata into the model.
writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)

{
  "name": "ImageSegmenter",
  "description": "Semantic image segmentation predicts whether each pixel of an image is associated with a certain class.",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be segmented.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  127.5
                ],
                "std": [
                  127.5
                ]
              }
            }
          ],
          "stats": {
            "max": [
              1.0
            ],
            "min": [
              -1.0
            ]
          }
        }
      ],
      "output_tensor_metadata": [
        {
  